## IMPORT


In [49]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
from langdetect import detect

# Carica il modello multilingua di SpaCy
try:
    nlp_it = spacy.load("it_core_news_sm")
    nlp_en = spacy.load("en_core_web_sm")
    print("Modelli SpaCy italiano e inglese caricati con successo")
    nlp_available = True
except Exception as e:
    print(f"ATTENZIONE: Errore nel caricamento dei modelli SpaCy: {e}")
    print("Installa con:")
    print("  python -m spacy download it_core_news_sm")
    print("  python -m spacy download en_core_web_sm")
    print("  pip install langdetect")
    nlp_available = False
    nlp_it = None
    nlp_en = None
    nlp = None

Modelli SpaCy italiano e inglese caricati con successo


## MODELLO RICONOSCIMENTO LINGUAGGIO


In [50]:
def get_nlp_model(text):
    """Rileva la lingua e restituisce il modello appropriato"""
    try:
        lang = detect(text[:500])  # Usa i primi 500 caratteri
        return nlp_it if lang == 'it' else nlp_en
    except:
        return nlp_it  # Default italiano

## DATASET LOADS


In [86]:
data = "./data/"
artists = pd.read_csv(f'{data}artists.csv', sep=',', index_col=0)
tracks = pd.read_csv(f'{data}tracks.csv', sep=',', index_col=0)

# ### Uncomment the following lines to backup at raw data files
# data = "./raw_data/"
# artists = pd.read_csv(f'{data}raw_artists.csv', sep=',', index_col=0)
# tracks = pd.read_csv(f'{data}raw_tracks.csv', sep=',', index_col=0)
# output_path = "./data/"
# artists.to_csv(f'{data}raw_artists.csv', sep=',')
# tracks.to_csv(f'{data}raw_tracks.csv', sep=',')

## ARTISTS


### DROP LONGITUDE, LATITUDE AND ACTIVE_END


In [87]:
# Print number of columns before dropping
print(f'Numero di colonne prima del drop: {len(artists.columns)}')
print(f'Colonne: {list(artists.columns)}')

columns_to_drop = ['longitude', 'latitude', 'active_end']
artists = artists.drop(columns=columns_to_drop)

# Print number of columns after dropping
print(f'\nNumero di colonne dopo il drop: {len(artists.columns)}')
print(f'Colonne rimosse: {columns_to_drop}')
print(f'Colonne rimanenti: {list(artists.columns)}')


Numero di colonne prima del drop: 13
Colonne: ['name', 'gender', 'birth_date', 'birth_place', 'nationality', 'description', 'active_start', 'active_end', 'province', 'region', 'country', 'latitude', 'longitude']

Numero di colonne dopo il drop: 10
Colonne rimosse: ['longitude', 'latitude', 'active_end']
Colonne rimanenti: ['name', 'gender', 'birth_date', 'birth_place', 'nationality', 'description', 'active_start', 'province', 'region', 'country']


## TRACK


### Drop no lyrics songs


In [88]:
# Drop tracks with null lyrics

# Statistiche prima del drop
initial_count = len(tracks)
null_lyrics_count = tracks['lyrics'].isna().sum()

tracks = tracks.dropna(subset=['lyrics'])

# Statistiche dopo il drop
final_count = len(tracks)
dropped_count = initial_count - final_count
print(f'\nTracce rimosse: {dropped_count}')
print(f'Tracce rimanenti: {final_count} ({(final_count/initial_count)*100:.2f}% del totale originale)')



Tracce rimosse: 3
Tracce rimanenti: 11163 (99.97% del totale originale)


### DROP FAKE LYRICS (2 PERCENTILE WITH LESS WORDS)


In [89]:
fake_lyrics1 = tracks[tracks['n_tokens'] < 65]
# fake_lyrics2 = tracks[tracks['n_sentences'] > 73]
fake_lyrics = pd.concat([fake_lyrics1])
fake_lyrics[['title','lyrics','n_tokens']].to_csv('./data/temp/fake_lyrics.csv', sep=',')
# Rimuovi le tracce fake dal dataframe tracks
tracks = tracks.drop(fake_lyrics.index)

print(f'Tracce rimosse: {fake_lyrics.shape[0]}')
print(f'Tracce rimanenti: {tracks.shape[0]}')

Tracce rimosse: 221
Tracce rimanenti: 10941


### FIX OUT OF RANGE YEARS OF THE TRACK


In [90]:
# Fix out-of-range years in tracks: impostare a vuoto se year < 1992 o > 2025
col = 'year'
total = len(tracks)

# Convertiamo in numerico
years = pd.to_numeric(tracks[col], errors='coerce')

# Identifichiamo valori fuori range
mask_out = (years < 1992) | (years > 2025)
out_count = int(mask_out.sum())
print(f'Valori fuori range (<1992 o >2025): {out_count} su {total} ({(out_count/total)*100:.2f}%)')

# Impostiamo a vuoto (pd.NA) i valori fuori range
tracks.loc[mask_out, col] = pd.NA

# Convertiamo la colonna in Int64 nullable per mantenere tipo numerico con NA
tracks[col] = pd.to_numeric(tracks[col], errors='coerce').astype('Int64')

# Statistiche dopo la pulizia
years_after = pd.to_numeric(tracks[col], errors='coerce')
print('\nStatistiche dopo la pulizia:')
print(years_after.describe())

# Numero di righe con year vuoto dopo la pulizia
final_empty = tracks[col].isna().sum()
print(f'\nNumero di righe con year vuoto dopo pulizia: {final_empty} su {total} ({(final_empty/total)*100:.2f}%)')


Valori fuori range (<1992 o >2025): 2113 su 10941 (19.31%)

Statistiche dopo la pulizia:
count         8414.0
mean     2015.268481
std         6.951416
min           1992.0
25%           2011.0
50%           2016.0
75%           2021.0
max           2025.0
Name: year, dtype: Float64

Numero di righe con year vuoto dopo pulizia: 2527 su 10941 (23.10%)


### FIX MISSING LYRICS STATISTICS


In [91]:
# ============================================================================
# FIX n_sentences
# ============================================================================
print("\n--- FIX n_sentences ---")
missing_before = tracks['n_sentences'].isna().sum()
print(f"Valori mancanti prima: {missing_before} ({(missing_before/len(tracks)*100):.2f}%)")

# Identifica righe con n_sentences mancante
mask_missing = tracks['n_sentences'].isna()

# Calcola n_sentences dalle lyrics
for idx in tracks[mask_missing].index:
    lyrics = tracks.loc[idx, 'lyrics']
    # Split su punti, esclamativi, interrogativi
    sentences = re.split(r'[.!?]+', str(lyrics))
    sentences = [s.strip() for s in sentences if s.strip()]
    n_sent = len(sentences)
    tracks.loc[idx, 'n_sentences'] = n_sent
    
# Statistiche dopo il fix
missing_after = tracks['n_sentences'].isna().sum()
fixed = missing_before - missing_after
print(f"Valori fixati: {fixed}")
print(f"Valori mancanti dopo: {missing_after} ({(missing_after/len(tracks)*100):.2f}%)")


# ============================================================================
# FIX n_tokens
# ============================================================================
print("\n--- FIX n_tokens ---")
missing_before = tracks['n_tokens'].isna().sum()
print(f"Valori mancanti prima: {missing_before} ({(missing_before/len(tracks)*100):.2f}%)")

# Identifica righe con n_tokens mancante
mask_missing = tracks['n_tokens'].isna()

# Calcola n_tokens dalle lyrics
for idx in tracks[mask_missing].index:
    lyrics = tracks.loc[idx, 'lyrics']
    # Estrai parole (token)
    tokens = re.findall(r'\b\w+\b', str(lyrics).lower())
    n_tok = len(tokens)
    tracks.loc[idx, 'n_tokens'] = n_tok
    
# Statistiche dopo il fix
missing_after = tracks['n_tokens'].isna().sum()
fixed = missing_before - missing_after
print(f"Valori fixati: {fixed}")
print(f"Valori mancanti dopo: {missing_after} ({(missing_after/len(tracks)*100):.2f}%)")


# ============================================================================
# FIX tokens_per_sent
# ============================================================================
print("\n--- FIX tokens_per_sent ---")
missing_before = tracks['tokens_per_sent'].isna().sum()
print(f"Valori mancanti prima: {missing_before} ({(missing_before/len(tracks)*100):.2f}%)")

# Identifica righe con tokens_per_sent mancante
mask_missing = tracks['tokens_per_sent'].isna()

# Calcola tokens_per_sent da n_tokens e n_sentences
for idx in tracks[mask_missing].index:
    n_tok = tracks.loc[idx, 'n_tokens']
    n_sent = tracks.loc[idx, 'n_sentences']
    
    if pd.notna(n_tok) and pd.notna(n_sent) and n_sent > 0:
        tok_per_sent = n_tok / n_sent
        tracks.loc[idx, 'tokens_per_sent'] = tok_per_sent
    
# Statistiche dopo il fix
missing_after = tracks['tokens_per_sent'].isna().sum()
fixed = missing_before - missing_after
print(f"Valori fixati: {fixed}")
print(f"Valori mancanti dopo: {missing_after} ({(missing_after/len(tracks)*100):.2f}%)")


# ============================================================================
# FIX char_per_tok
# ============================================================================
print("\n--- FIX char_per_tok ---")
missing_before = tracks['char_per_tok'].isna().sum()
print(f"Valori mancanti prima: {missing_before} ({(missing_before/len(tracks)*100):.2f}%)")

# Identifica righe con char_per_tok mancante
mask_missing = tracks['char_per_tok'].isna()

# Calcola char_per_tok dalle lyrics
for idx in tracks[mask_missing].index:
    lyrics = tracks.loc[idx, 'lyrics']
    # Estrai token e calcola media caratteri
    tokens = re.findall(r'\b\w+\b', str(lyrics).lower())
    n_tok = len(tokens)
    
    if n_tok > 0:
        total_chars = sum(len(t) for t in tokens)
        char_per_tok = total_chars / n_tok
        tracks.loc[idx, 'char_per_tok'] = char_per_tok
    
# Statistiche dopo il fix
missing_after = tracks['char_per_tok'].isna().sum()
fixed = missing_before - missing_after
print(f"Valori fixati: {fixed}")
print(f"Valori mancanti dopo: {missing_after} ({(missing_after/len(tracks)*100):.2f}%)")


# ============================================================================
# FIX lexical_density
# ============================================================================
print("\n--- FIX lexical_density ---")
missing_before = tracks['lexical_density'].isna().sum()
print(f"Valori mancanti prima: {missing_before} ({(missing_before/len(tracks)*100):.2f}%)")

# Identifica righe con lexical_density mancante
mask_missing = tracks['lexical_density'].isna()

# Calcola lexical_density dalle lyrics
for idx in tracks[mask_missing].index:
    lyrics = tracks.loc[idx, 'lyrics']
    # Estrai token e calcola densità lessicale
    tokens = re.findall(r'\b\w+\b', str(lyrics).lower())
    n_tok = len(tokens)
    
    if n_tok > 0:
        unique_tokens = len(set(tokens))
        lex_dens = unique_tokens / n_tok
        tracks.loc[idx, 'lexical_density'] = lex_dens
    
# Statistiche dopo il fix
missing_after = tracks['lexical_density'].isna().sum()
fixed = missing_before - missing_after
print(f"Valori fixati: {fixed}")
print(f"Valori mancanti dopo: {missing_after} ({(missing_after/len(tracks)*100):.2f}%)")


# ============================================================================
# FIX avg_token_per_clause
# ============================================================================
print("\n--- FIX avg_token_per_clause ---")

if not nlp_available or nlp_it is None or nlp_en is None:
    print("ERRORE: Modelli SpaCy non disponibili. Salta il calcolo di avg_token_per_clause")
else:
    missing_before = tracks['avg_token_per_clause'].isna().sum() if 'avg_token_per_clause' in tracks.columns else len(tracks)
    print(f"Valori mancanti prima: {missing_before} ({(missing_before/len(tracks)*100):.2f}%)")
    
    if 'avg_token_per_clause' not in tracks.columns:
        tracks['avg_token_per_clause'] = pd.NA
        print("Colonna 'avg_token_per_clause' creata")
    
    mask_missing = tracks['avg_token_per_clause'].isna()
    indices_to_process = tracks[mask_missing].index.tolist()
    total_to_process = len(indices_to_process)
    
    print(f"Elaborazione di {total_to_process} righe con SpaCy...")
    
    # Prepara i dati: [(idx, lyrics, lang), ...]
    data_to_process = []
    for idx in indices_to_process:
        lyrics = tracks.loc[idx, 'lyrics']
        if pd.notna(lyrics) and lyrics != '':
            try:
                lang = detect(str(lyrics)[:500])
            except:
                lang = 'it'
            data_to_process.append((idx, str(lyrics)[:1000000], lang))
    
    # Elabora in batch per lingua
    print("Elaborazione canzoni italiane...")
    italian_data = [(idx, text) for idx, text, lang in data_to_process if lang == 'it']
    for i, (idx, lyrics) in enumerate(italian_data):
        try:
            doc = nlp_it(lyrics)
            n_clauses = sum(1 for _ in doc.sents)
            for sent in doc.sents:
                for token in sent:
                    if token.pos_ == 'VERB' and token.dep_ in ['ccomp', 'xcomp', 'advcl', 'relcl', 'acl', 'csubj', 'csubjpass']:
                        n_clauses += 1
            n_tokens = len([t for t in doc if t.is_alpha])
            if n_clauses > 0:
                tracks.loc[idx, 'avg_token_per_clause'] = n_tokens / n_clauses
            
            if (i + 1) % 500 == 0:
                print(f"  Processate {i + 1}/{len(italian_data)} canzoni italiane...")
        except:
            continue
    
    print("Elaborazione canzoni inglesi...")
    english_data = [(idx, text) for idx, text, lang in data_to_process if lang != 'it']
    for i, (idx, lyrics) in enumerate(english_data):
        try:
            doc = nlp_en(lyrics)
            n_clauses = sum(1 for _ in doc.sents)
            for sent in doc.sents:
                for token in sent:
                    if token.pos_ == 'VERB' and token.dep_ in ['ccomp', 'xcomp', 'advcl', 'relcl', 'acl', 'csubj', 'csubjpass']:
                        n_clauses += 1
            n_tokens = len([t for t in doc if t.is_alpha])
            if n_clauses > 0:
                tracks.loc[idx, 'avg_token_per_clause'] = n_tokens / n_clauses
            
            if (i + 1) % 500 == 0:
                print(f"  Processate {i + 1}/{len(english_data)} canzoni inglesi...")
        except:
            continue
    
    missing_after = tracks['avg_token_per_clause'].isna().sum()
    fixed = missing_before - missing_after
    print(f"\nValori fixati: {fixed}")
    print(f"Valori mancanti dopo: {missing_after} ({(missing_after/len(tracks)*100):.2f}%)")

tracks.to_csv('./data/temp/tracks_cleaned.csv', sep=',')


--- FIX n_sentences ---
Valori mancanti prima: 73 (0.67%)
Valori fixati: 73
Valori mancanti dopo: 0 (0.00%)

--- FIX n_tokens ---
Valori mancanti prima: 73 (0.67%)
Valori fixati: 73
Valori mancanti dopo: 0 (0.00%)

--- FIX tokens_per_sent ---
Valori mancanti prima: 73 (0.67%)
Valori fixati: 73
Valori mancanti dopo: 0 (0.00%)

--- FIX char_per_tok ---
Valori mancanti prima: 73 (0.67%)
Valori fixati: 73
Valori mancanti dopo: 0 (0.00%)

--- FIX lexical_density ---
Valori mancanti prima: 73 (0.67%)
Valori fixati: 73
Valori mancanti dopo: 0 (0.00%)

--- FIX avg_token_per_clause ---
Valori mancanti prima: 73 (0.67%)
Elaborazione di 73 righe con SpaCy...
Elaborazione canzoni italiane...
Elaborazione canzoni inglesi...

Valori fixati: 73
Valori mancanti dopo: 0 (0.00%)


### CHANGE STRINGS IN FEATURED_ARTISTS TO LIST


In [92]:
# === CONVERSIONE DELLA COLONNA 'featured_artists' IN LISTE DI STRINGHE ===

def process_featured_artists(featured_str):
    """Converte una stringa di artisti separati da virgola in una lista pulita"""
    # Se è già una lista, la restituisco così com’è
    if isinstance(featured_str, list):
        return featured_str
    
    # Se è NaN o vuoto → lista vuota
    if pd.isna(featured_str) or featured_str == '':
        return []

    # Converte in stringa (per sicurezza)
    featured_str = str(featured_str)
    # Rimuove caratteri invisibili Unicode
    featured_str = featured_str.replace('\u200b', '').replace('\xa0', ' ').replace('\u202f', ' ')
    # Splitta per virgola e pulisce spazi
    artists_list = [artist.strip() for artist in featured_str.split(',')]
    # Rimuove stringhe vuote
    return [artist for artist in artists_list if artist]

# Applica la conversione DIRETTAMENTE al DataFrame originale
tracks['featured_artists'] = tracks['featured_artists'].apply(process_featured_artists)

# Crea un dataframe temporaneo solo per ispezione e salvataggio
feat_tracks = tracks[tracks['featured_artists'].apply(len) > 0].copy()

# Salva solo le tracce con featured artists (per controllo o analisi)
tracks.to_csv(f'{data}/temp/tracks_cleaned.csv', index=False)

# Log di controllo
print(f"\n=== CONVERSIONE 'featured_artists' COMPLETATA ===")
print(f"Totale tracce con featured artists: {len(feat_tracks)}")
print(f"Tracce senza featured artists: {(tracks['featured_artists'].apply(len) == 0).sum()}")
print("\nEsempi di conversione:")
print(feat_tracks[['title', 'featured_artists']].head())



=== CONVERSIONE 'featured_artists' COMPLETATA ===
Totale tracce con featured artists: 3477
Tracce senza featured artists: 7464

Esempi di conversione:
                 title       featured_artists
id                                           
TR934808  ​polka 2 :-/           [Ernia, Guè]
TR760029         POLKA        [Thelonious B.]
TR916821  ​britney ;-)  [MamboLosco, RADICAL]
TR480968           CEO               [Taxi B]
TR585039        LONDRA                [Rkomi]


### FIX THE SONGS WITHOUT FEATURED ARTIST IN THE SPECIFIC COLUMN BUT WITH FEAT ARTIST IN THE TITLE


In [93]:
tracks['featured_artists'] = tracks['featured_artists'].apply(
    lambda x: x if isinstance(x, list) else ([] if pd.isna(x) else [x])
)

tracks_no_explicit_feat = tracks[tracks['featured_artists'].apply(len) == 0]
print(f"\nTotale tracce senza featured artists nella colonna: {len(tracks_no_explicit_feat)}")

has_amp_after_by = tracks_no_explicit_feat['full_title'].str.contains(r'\bby\b[^&]*&', case=False, na=False, regex=True)

tracks_by_amp = tracks_no_explicit_feat[has_amp_after_by].copy()

# Conta quanti "&" ci sono dopo "by"
def count_ampersands_after_by(title):
    """Conta quanti '&' ci sono dopo 'by' nel titolo"""
    match = re.search(r'\bby\b(.+)', title, re.IGNORECASE)
    if match:
        after_by = match.group(1)
        return after_by.count('&')
    return 0

tracks_by_amp['n_ampersands'] = tracks_by_amp['full_title'].apply(count_ampersands_after_by)


# Creazione dataframe featuring estratti da "by nome & nome2 & nome3"

def extract_featuring_names(title):
    """Estrae i nomi degli artisti dopo 'by' separati da '&'"""
    match = re.search(r'\bby\b\s*(.+)', title, re.IGNORECASE)
    if match:
        after_by = match.group(1).strip()
        # Pulisce eventuali parentesi finali e altri caratteri
        after_by = re.sub(r'\s*[\(\[\]].*$', '', after_by)
        # Splitta per '&' e pulisce gli spazi
        names = [name.strip() for name in after_by.split('&')]
        return names
    return []

# Applica l'estrazione solo alle tracce con & dopo by
tracks_by_amp['featuring_list'] = tracks_by_amp['full_title'].apply(extract_featuring_names)



# Gestisco il caso in cui si hanno feat scritti nel modo: by Lazza, Murda, Beny Jr, Guy2Bezbar & Elias
# attualmente nel vettore dei feature prendo quello a destra del & fino al by e quello dopo il &
# esempio: ['Lazza, Murda, Beny Jr, Guy2Bezbar', 'Elias']
# devo ridividere quelli associati con la , in elementi singoli del vettore

# Splitta anche per virgole oltre che per &
def split_featuring_names(names_list):
    """Splitta ulteriormente i nomi che contengono virgole"""
    final_names = []
    for name in names_list:
        # Se il nome contiene virgole, splittalo
        if ',' in name:
            # Splitta per virgola e aggiungi ogni pezzo
            final_names.extend([n.strip() for n in name.split(',')])
        else:
            final_names.append(name.strip())
    return final_names

# Applica lo split delle virgole
tracks_by_amp['featuring_list'] = tracks_by_amp['featuring_list'].apply(split_featuring_names)


# Crea dataframe con una riga per ogni combinazione canzone-featuring
feat_data = []
for idx, row in tracks_by_amp.iterrows():
    names = row['featuring_list']
    if len(names) > 0:
        feat_data.append({
            'track_id': idx,
            'name_artist': row['name_artist'],
            'full_title': row['full_title'],
            'featuring_names': names,
            'n_featuring': len(names)
        })

# levo dai vettori nella colonna featuring_names le eventuali occorrenze del name_artist
# Normalizza gli spazi nei featuring (rimuove \xa0 e altri spazi speciali)
for entry in feat_data:
    artist_name_lower = entry['name_artist'].strip().lower()
    # Normalizza solo i nomi nei featuring e confronta con l'artista principale
    entry['featuring_names'] = [
        name.replace('\xa0', ' ').strip() for name in entry['featuring_names'] 
        if name.replace('\xa0', ' ').strip().lower() != artist_name_lower
    ]

df_featuring = pd.DataFrame(feat_data)

print(f"\n=== DATAFRAME FEATURING ESTRATTI ===\n")
print(f"Totale tracce con featuring estratti: {len(df_featuring)}")


# Salva il dataframe (opzionale)
# df_featuring.to_csv('./data/temp/feats.csv', index=False)

# === AGGIORNA IL DATAFRAME ORIGINALE CON I NUOVI FEATURING ESTRATTI ===

tracks['featured_artists'] = tracks['featured_artists'].apply(
    lambda x: x if isinstance(x, list) else ([] if pd.isna(x) else [x])
)

# Series con track_id → lista featuring
feat_map = df_featuring.set_index('track_id')['featuring_names']

# Aggiorna solo le righe i cui indici sono presenti in feat_map
tracks.loc[feat_map.index, 'featured_artists'] = feat_map

tracks.to_csv('./data/temp/tracks_cleaned.csv', sep=',')




Totale tracce senza featured artists nella colonna: 7464

=== DATAFRAME FEATURING ESTRATTI ===

Totale tracce con featuring estratti: 406


### FIX SOME ACTIVE_START IN THE DATASET ARTIST TAKING FIRST SONG OR FIRST FEAT


In [98]:
# Copia del dataframe artists
artists_compare = artists.copy()

# Converto active_start e birth_date in anno intero
artists_compare['active_start'] = pd.to_datetime(
    artists['active_start'], errors='coerce'
).dt.year
artists_compare['birth_date'] = pd.to_datetime(
    artists['birth_date'], errors='coerce'
).dt.year

# Assicurati che 'year' sia numerico
tracks['year'] = pd.to_numeric(tracks['year'], errors='coerce')

# Esplodi la colonna featured_artists (se è già lista)
exploded = tracks.explode('featured_artists').rename(columns={'featured_artists': 'featured_artist'})

# Rimuovi eventuali NaN
exploded = exploded[exploded['featured_artist'].notna()]

# Normalizza i nomi
exploded['featured_artist'] = exploded['featured_artist'].str.strip().str.lower()
artists_name_lower = artists_compare['name'].str.lower()

# Trova il primo anno come artista principale
main_years = tracks.groupby('id_artist')['year'].min()

# Trova il primo anno come artista featured
featured_years = exploded.groupby('featured_artist')['year'].min()

# Mappa gli anni principali e featured agli artisti
main_years_mapped = pd.Series(artists_compare.index.map(main_years), index=artists_compare.index)
featured_years_mapped = pd.Series(artists_name_lower.map(featured_years), index=artists_compare.index)

# Combina e riempi active_start
artists_compare['active_start'] = artists_compare['active_start'].fillna(
    main_years_mapped.combine_first(featured_years_mapped)
)

# Converti in intero
artists_compare['active_start'] = artists_compare['active_start'].astype('Int64')

# Funzione per identificare valori "sbagliati"
def is_bad(row):
    if pd.isna(row['active_start']) or pd.isna(row['birth_date']):
        return False
    return row['active_start'] <= row['birth_date']

# Analisi prima e dopo
artists_before = artists.copy()
artists_before['active_start'] = pd.to_datetime(artists_before['active_start'], errors='coerce').dt.year
artists_before['birth_date'] = pd.to_datetime(artists_before['birth_date'], errors='coerce').dt.year
artists_before['bad'] = artists_before.apply(is_bad, axis=1)
artists_compare['bad'] = artists_compare.apply(is_bad, axis=1)

num_bad_before = artists_before['bad'].sum()
num_bad_after = artists_compare['bad'].sum()
new_bad = ((~artists_before['bad']) & (artists_compare['bad'])).sum()
old_bad_still_bad = ((artists_before['bad']) & (artists_compare['bad'])).sum()
good_still_good = ((~artists_before['bad']) & (~artists_compare['bad'])).sum()

total = len(artists)

# Conteggio righe che hanno ricevuto un active_start imputato
missing_before = pd.to_datetime(artists['active_start'], errors='coerce').isna()
filled_now = missing_before & artists_compare['active_start'].notna()
num_imputed = filled_now.sum()

print(f"Bad rows before update: {num_bad_before}")
print(f"Bad rows after update:  {num_bad_after}")
print(f"Newly bad rows introduced: {new_bad}")
print(f"Old bad rows still bad:    {old_bad_still_bad}")
print(f"Good rows still good:      {good_still_good}")
print(f"Percentage of dataset that became bad: {100 * new_bad / total:.2f}%")
print(f"Percentage of dataset that remains bad: {100 * old_bad_still_bad / total:.2f}%")
print(f"Righe che hanno ricevuto un active_start imputato: {num_imputed}")
print(f"\nGli artisti rimasti senza active_start valido dopo l'imputazione sono:\n {artists_compare.loc[artists_compare['active_start'].isna(), 'name']}")

# === APPLICA LE MODIFICHE AL DATASET ARTISTS ORIGINALE ===
print("\n=== AGGIORNAMENTO DATASET ARTISTS ===")

# Converti active_start in formato datetime con anno (mantieni il formato originale)
# Usa il formato 'YYYY-01-01' per mantenere la compatibilità
artists['active_start'] = artists_compare['active_start'].apply(
    lambda x: pd.Timestamp(year=int(x), month=1, day=1) if pd.notna(x) else pd.NaT
)

print(f"Dataset artists aggiornato con {num_imputed} nuovi valori di active_start")
print(f"Tipo di dato active_start: {artists['active_start'].dtype}")

# Salva il dataframe finale (temporaneo)
artists.to_csv('./data/temp/artists_cleaned.csv', sep=',', index=True)


Bad rows before update: 0
Bad rows after update:  0
Newly bad rows introduced: 0
Old bad rows still bad:    0
Good rows still good:      104
Percentage of dataset that became bad: 0.00%
Percentage of dataset that remains bad: 0.00%
Righe che hanno ricevuto un active_start imputato: 0

Gli artisti rimasti senza active_start valido dopo l'imputazione sono:
 id_author
ART42220690    o zulù
Name: name, dtype: object

=== AGGIORNAMENTO DATASET ARTISTS ===
Dataset artists aggiornato con 0 nuovi valori di active_start
Tipo di dato active_start: datetime64[ns]


### CHECK IF ALL THE ARTISTS HAVE AT LEAST ONE SONG IN THE TRACKS DATASET AFTER THE CLEANING


In [108]:
def check_artists_have_tracks(artists_df, tracks_df):
    """
    Controlla quali artisti hanno almeno una traccia nel dataset tracks.
    Aggiorna direttamente il dataframe artists_df aggiungendo una colonna 'has_tracks'.
    Controlla sia come artista principale (id_artist) che come featuring (featured_artists).
    """
    print("=== Controllo artisti con tracce associate ===")
    total_artists = len(artists_df)
    print(f"Totale artisti nel dataset: {total_artists}")

    # Set degli ID artisti presenti in tracks come artisti principali
    main_artists = set(tracks_df['id_artist'].dropna().unique())
    
    # Set dei nomi artisti presenti in tracks come featuring (esplodendo le liste)
    feat_artists = set()
    for feat_list in tracks_df['featured_artists'].dropna():
        if isinstance(feat_list, list):
            feat_artists.update([name.strip().lower() for name in feat_list])
    
    # Mappa nome → id artista
    name_to_id = dict(zip(artists_df['name'].str.lower(), artists_df.index))
    
    # ID degli artisti che appaiono come featuring
    feat_artist_ids = {name_to_id[name] for name in feat_artists if name in name_to_id}
    
    # Combina artisti principali e featuring
    artists_with_tracks = main_artists | feat_artist_ids
    
    print(f"Totale artisti con almeno una traccia (main): {len(main_artists)}")
    print(f"Totale artisti che appaiono come featuring: {len(feat_artist_ids)}")
    print(f"Totale artisti con almeno una traccia (main + featuring): {len(artists_with_tracks)}")

    # Crea colonna booleana 'has_tracks'
    artists_df['has_tracks'] = artists_df.index.isin(artists_with_tracks)

    # Conta quanti artisti hanno tracce e quanti no
    num_with_tracks = artists_df['has_tracks'].sum()
    num_without_tracks = total_artists - num_with_tracks
    print(f"Artisti con almeno una traccia: {num_with_tracks}")
    print(f"Artisti senza tracce: {num_without_tracks}")

    # Mostra alcuni esempi di artisti senza tracce
    if num_without_tracks > 0:
        print("\nEsempi di artisti senza tracce associate:")
        print(artists_df.loc[~artists_df['has_tracks'], ['name']].head(10))

    print("=== Controllo completato ===\n")
    return artists_df

# Esegui il controllo
artists = check_artists_have_tracks(artists, tracks)

=== Controllo artisti con tracce associate ===
Totale artisti nel dataset: 104
Totale artisti con almeno una traccia (main): 104
Totale artisti che appaiono come featuring: 85
Totale artisti con almeno una traccia (main + featuring): 104
Artisti con almeno una traccia: 104
Artisti senza tracce: 0
=== Controllo completato ===



## SAVE CHANGES ON CSV


### ARTISTS

    Controlla quali artisti hanno almeno una traccia nel dataset tracks.    Controlla sia come artista principale (id_artist) che come featuring (featured_artists).    # Set degli ID artisti presenti in tracks come artisti principali    main_artists = set(tracks_df['id_artist'].dropna().unique())        # Set dei nomi artisti presenti in tracks come featuring (esplodendo le liste)    feat_artists = set()    for feat_list in tracks_df['featured_artists'].dropna():        if isinstance(feat_list, list):            feat_artists.update([name.strip().lower() for name in feat_list])        # Mappa nome → id artista    name_to_id = dict(zip(artists_df['name'].str.lower(), artists_df.index))        # ID degli artisti che appaiono come featuring    feat_artist_ids = {name_to_id[name] for name in feat_artists if name in name_to_id}        # Combina artisti principali e featuring    artists_with_tracks = main_artists | feat_artist_ids        print(f"Totale artisti con almeno una traccia (main): {len(main_artists)}")    print(f"Totale artisti che appaiono come featuring: {len(feat_artist_ids)}")    print(f"Totale artisti con almeno una traccia (main + featuring): {len(artists_with_tracks)}")    # Crea colonna booleana 'has_tracks'
    artists_df['has_tracks'] = artists_df.index.isin(artists_with_tracks)

    # Conta quanti artisti hanno tracce e quanti no
    num_with_tracks = artists_df['has_tracks'].sum()
    num_without_tracks = total_artists - num_with_tracks
    print(f"Artisti con almeno una traccia: {num_with_tracks}")
    print(f"Artisti senza tracce: {num_without_tracks}")

    # Mostra alcuni esempi di artisti senza tracce
    if num_without_tracks > 0:
        print("\nEsempi di artisti senza tracce associate:")
        print(artists_df.loc[~artists_df['has_tracks'], ['name']].head(10))

    print("=== Controllo completato ===\n")
    return artists_df

# Esegui il controllo

artists = check_artists_have_tracks(artists, tracks)


In [8]:
# Save artists dataset
# output_path = f'{data}artists.csv'
# artists.to_csv(output_path, sep=';')
# print(f'Dataset tracks salvato in: {output_path}')


### TRACKS


In [ ]:
# Save tracks dataset
output_path = f'{data}tracks.csv'
tracks.to_csv(output_path, sep=',')
print(f'Dataset tracks salvato in: {output_path}')


Dataset tracks salvato in: ./data/cleaned.csv
